# Equation of state (EOS) WorkTree

To run this tutorial, you need to install aiida-worktree and restart the daemon. Open a terminal and run:
```console
pip install aiida-worktree
verdi daemon restart
```

## Create the calcfunction node

In [3]:
from aiida import orm
from aiida_worktree import node

# explicitly define the output socket name to match the return value of the function
@node.calcfunction(outputs=[["General", "structures"]])
def scale_structure(structure, scales):
    """Scale the structure by the given scales."""
    atoms = structure.get_ase()
    structures = {}
    for i in range(len(scales)):
        atoms1 = atoms.copy()
        atoms1.set_cell(atoms.cell * scales[i], scale_atoms=True)
        structure = orm.StructureData(ase=atoms1)
        structures[f"s_{i}"] = structure
    return {"structures": structures}

# Output result from ctx to the output socket
@node.group(outputs=[["ctx.result", "result"]])
def all_scf(structures, scf_inputs):
    """Run the scf calculation for each structure."""
    from aiida_worktree import WorkTree, build_node
    # register PwCalculation calcjob as a node class
    PwCalculation = build_node({"path": "aiida_quantumespresso.calculations.pw.PwCalculation"})
    wt = WorkTree()
    for key, structure in structures.items():
        pw1 = wt.nodes.new(PwCalculation, name=f"pw1_{key}", structure=structure)
        pw1.set(scf_inputs)
        # save the output parameters to the ctx
        pw1.to_ctx = [["output_parameters", f"result.{key}"]]
    return wt


@node.calcfunction()
# because this is a calcfunction, and the input datas are dynamic, we need use **datas.
def eos(**datas):
    """Fit the EOS of the data."""
    from ase.eos import EquationOfState

    volumes = []
    energies = []
    for _, data in datas.items():
        volumes.append(data.dict.volume)
        energies.append(data.dict.energy)
        unit = data.dict.energy_units
    #
    eos = EquationOfState(volumes, energies)
    v0, e0, B = eos.fit()
    eos = orm.Dict({"unit": unit, "v0": v0, "e0": e0, "B": B})
    return eos

## Build the worktree
Three steps:

- create an empty WorkTree
- add nodes: scale_structure, all_scf and eos.
- link the output and input sockets for the nodes.

### Visualize the worktree
If you are running in a Jupiter notebook, you can visualize the worktree directly.

In [4]:
from aiida_worktree import WorkTree

wt = WorkTree("eos")
scale_structure1 = wt.nodes.new(scale_structure, name="scale_structure1")
all_scf1 = wt.nodes.new(all_scf, name="all_scf1")
eos1 = wt.nodes.new(eos, name="eos1")
wt.links.new(scale_structure1.outputs["structures"], all_scf1.inputs["structures"])
wt.links.new(all_scf1.outputs["result"], eos1.inputs["datas"])
wt

NodeGraphWidget(value={'name': 'eos', 'uuid': '9af02124-fcd8-11ee-90f5-02425c27bd68', 'state': 'CREATED', 'nod…

## Prepare inputs and run


In [5]:
from aiida import load_profile
from aiida.orm import Dict, KpointsData, StructureData, load_code, load_group
from ase.build import bulk

load_profile()

si = orm.StructureData(ase=bulk("Si"))
code = load_code("qe-7.2-pw@localhost")
pw_paras = Dict({
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 30,
            "ecutrho": 240,
            "occupations": "smearing",
            "smearing": "gaussian",
            "degauss": 0.1,
        },
    })
# Load the pseudopotential family.
pseudo_family = load_group('SSSP/1.3/PBEsol/efficiency')
pseudos = pseudo_family.get_pseudos(structure=si)
#
metadata = {'options': {
                'resources': {
                'num_machines': 1,
                'num_mpiprocs_per_machine': 1,
            },
        }
    }

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
pseudos = pseudo_family.get_pseudos(structure=si)
scf_inputs = {"code": code,
        "parameters": pw_paras,
        "kpoints": kpoints,
        "pseudos": pseudos,
        "metadata": metadata
        }
#-------------------------------------------------------
# set the input parameters for each node
wt.nodes["scale_structure1"].set({"structure": si, "scales": [0.95, 1.0, 1.05]})
wt.nodes["all_scf1"].set({"scf_inputs": scf_inputs})
print("Waiting for the worktree to finish...")
wt.submit(wait=True, timeout=300)
# one can also run the worktree directly
# wt.run()



WorkTree node created, PK: 10919


Print out the results:

In [6]:
data = wt.nodes["eos1"].outputs["result"].value.get_dict()
print('B: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**data))

B: 0.53596259211164
v0: 41.134100879971
e0: -308.19240692174
v0: 41.134100879971


## Use node group
The Node Group allow user to create a dynamic workflow based on the input value, as well as nested workflows.

In [7]:
from aiida_worktree import WorkTree, node

@node.group(outputs=[["eos1.result", "result"]])
def eos_worktree(structure=None, scales=None, scf_inputs=None):
    wt = WorkTree("eos")
    scale_structure1 = wt.nodes.new(scale_structure, name="scale_structure1", structure=structure, scales=scales)
    all_scf1 = wt.nodes.new(all_scf, name="all_scf1", scf_inputs=scf_inputs)
    eos1 = wt.nodes.new(eos, name="eos1")
    wt.links.new(scale_structure1.outputs["structures"], all_scf1.inputs["structures"])
    wt.links.new(all_scf1.outputs["result"], eos1.inputs["datas"])
    return wt

Then we can use the `eos_worktree` in two ways:

- Direct run the function and generate the worktree, then submit
- Use it as a node inside another worktree to create nested workflow.

### Use the node group directly

In [8]:
wt = eos_worktree(structure=si, scales=[0.95, 1.0, 1.05], scf_inputs=scf_inputs)
display(wt)
print("Waiting for the worktree to finish...")
wt.submit(wait=True, timeout=300)
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**wt.nodes["eos1"].outputs["result"].value.get_dict()))

NodeGraphWidget(value={'name': 'eos', 'uuid': 'b190d75c-fcd8-11ee-90f5-02425c27bd68', 'state': 'CREATED', 'nod…

Waiting for the worktree to finish...
WorkTree node created, PK: 10948

Result: 
B: 0.53596259211164
v0: 41.134100879971
e0: -308.19240692174
v0: 41.134100879971


### Use it inside another worktree
For example, we want to combine relax with eos.

In [11]:
from aiida_worktree import WorkTree, build_node
from copy import deepcopy
# register PwCalculation calcjob as a node class
PwCalculation = build_node({"path": "aiida_quantumespresso.calculations.pw.PwCalculation"})

#-------------------------------------------------------
relax_pw_paras = deepcopy(pw_paras)
relax_pw_paras["CONTROL"]["calculation"] = "vc-relax"
relax_inputs = {
        "structure": si,
        "code": code,
        "parameters": relax_pw_paras,
        "kpoints": kpoints,
        "pseudos": pseudos,
        "metadata": metadata
        }
#-------------------------------------------------------
wt = WorkTree("relax_eos")
relax_node = wt.nodes.new(PwCalculation, name="relax1")
relax_node.set(relax_inputs)
eos_wt_node = wt.nodes.new(eos_worktree, name="eos1", scales=[0.95, 1.0, 1.05], scf_inputs=scf_inputs)
wt.links.new(relax_node.outputs["output_structure"], eos_wt_node.inputs["structure"])
#-------------------------------------------------------
display(wt)
print("Waiting for the worktree to finish...")
wt.submit(wait=True, timeout=300)
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**wt.nodes["eos1"].outputs["result"].value.get_dict()))

NodeGraphWidget(value={'name': 'relax_eos', 'uuid': 'e627c526-fcd7-11ee-9f57-02425c27bd68', 'state': 'CREATED'…

Waiting for the worktree to finish...
WorkTree node created, PK: 10878

Result: 
B: 0.51887865581152
v0: 41.167742940863
e0: -308.19005654623
v0: 41.167742940863


## Summary
There are many ways to create the workflow using node group. For example, one can add the relax step inside the `eos_worktree`, and add a `run_relax` argument to control the logic.